In [23]:
import requests
from urllib.parse import urlencode

## Requesting entities from wikipedia

https://en.wikipedia.org/w/api.php?action=opensearch&format=json&redirects=resolve&origin=*&search=man

In [138]:
wiki_url_pattern = "https://en.wikipedia.org/w/api.php?action=opensearch&format=json&redirects=resolve&origin=*&search={}"

def get_entity(name):
    query_url = wiki_url_pattern.format(name)
    r = requests.get(query_url)
    if r.status_code != 200:
        return
    r_js = r.json()
#     print(r_js[1][0])
    return  r_js[1][0], r_js[2][0], r_js[3][0]

In [83]:
entity_name, entity_description, wiki_url = get_entity("man")

In [65]:
entity_name

'Man'

In [66]:
entity_description

'A man is a male human. The term man is usually reserved for an adult male, with the term boy being the usual term for a male child or adolescent.'

In [84]:
wiki_url

'https://en.wikipedia.org/wiki/Man'

## Getting the associations from Jörn's API

```
curl 'https://projects.dfki.uni-kl.de/graph-pattern-learner/api/predict' -H 'Origin: https://joernhees.github.io' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-GB,en;q=0.9,de;q=0.8' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36' -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'Accept: application/json, text/javascript, */*; q=0.01' -H 'Referer: https://joernhees.github.io/associations/gp_learner/demo/predict.html' -H 'Connection: keep-alive' --data 'source=http%3A%2F%2Fdbpedia.org%2Fresource%2FMan' --compressed
```

```
POST /graph-pattern-learner/api/predict HTTP/1.1
Host: projects.dfki.uni-kl.de
Connection: keep-alive
Content-Length: 48
Accept: application/json, text/javascript, */*; q=0.01
Origin: https://joernhees.github.io
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Referer: https://joernhees.github.io/associations/gp_learner/demo/predict.html
Accept-Encoding: gzip, deflate, br
Accept-Language: en-GB,en;q=0.9,de;q=0.8
```

In [25]:
urlencode({"source":entity_uri})

'source=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FMan'

In [136]:
association_url = "https://projects.dfki.uni-kl.de/graph-pattern-learner/api/predict"
dbpedia_pattern = "http://dbpedia.org/resource/{}"

def get_prediction(entity_name):
    no_spaces_name = "_".join(entity_name.split())
    dbpedia_url = dbpedia_pattern.format(no_spaces_name)
    payload = {'source': dbpedia_url}
    r = requests.post(association_url, data=payload)
    if r.status_code != 200:
        return r
    
    r_js = r.json()
    return r_js

In [75]:
r = get_prediction("Banana")
r['fused_results']['f_measures'][0][0]

'http://dbpedia.org/resource/Fruit'

## putting it together

In [120]:
def get_association(query):
    entity_name, entity_description, wiki_url = get_entity(query)
    r = get_prediction(entity_name)
    prediction_uri = r['fused_results']['precisions'][0][0]
    prediction_name = prediction_uri.split('/')[-1]
    return entity_name, prediction_name

In [121]:
pred = get_association('man')
pred

('Man', 'Human')

## Output

In [122]:
import subprocess

In [123]:
def say(sentence):
    subprocess.run(['say', sentence])

In [124]:
say("hi sandro")

## Everything

In [141]:
say_pattern = "When I see {}, I think of {}"

def see_say(query):
    db_query, response = get_association(query)
    say_sentence = say_pattern.format(db_query, response)
    print(say_sentence)
    say(say_sentence)

In [154]:
see_say('leggings')

When I see Leggings, I think of Trousers
